# Hello OpenQASM3
#### @Quantom  & @Micheal Healy & @Hoss Ajallooiean - Qiskit Slack
#### @taalexander & @mbhealy & @katetsu - Github

Join us on the Qiskit Slack channel [#ieee-tutorial-openqasm3](https://qiskit.slack.com/archives/C02HQLDS3D3)


We'll be using Qiskit throughout this tutorial. As OpenQASM3 is still in alpha support this tutorial requires some special branches to be installed to access supporting hardware. This notebook will guide your through getting setup.

Don't hesitate to ask questions either in the Zoom call or the [Slack channel](https://qiskit.slack.com/archives/C02HQLDS3D3).

### Preamble

In [1]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

### Installation

The tutorial consists of three notebooks: 
- This Hello World notebook
- The [presentation notebook](./OpenQASM3-Presentation.ipynb)
- The [exercise notebook](./OpenQASM3-Exercise-IPE.ipynb) 

There are also a number of ancillary Python files and images. The notebooks can be run locally or through the [Quantum Lab](https://lab.quantum-computing.ibm.com/) although some functionality will be missing on the lab as it is using older library versions.

Please follow the installation guide below to get started.

### IBM Quantum Services Registration

To get going you'll need to register an account with IBM quantum services if you don't already have one:
- [Sign up for an IBM Quantum account](https://quantum-computing.ibm.com/)
- After registration reach out to the Tutorial presenters in the Zoom chat or Qiskit Slack channel with your email so that the presenters may give you access to the hardware for this tutorial.

## Notebook Access

### Local Machine

Instructions:
- Install Qiskit by following these [instructions](https://qiskit.org/documentation/install.html).
    - Setup a Python virtual environment for the tutorial (good practice but not necessary).
    - Make sure you have either Qiskit 0.31.0 installed. For full tutorial functionality install Qiskit's `main` branch:
        - `pip install qiskit==0.31.0 jupyter matplotlib scipy`
        - `git clone git@github.com:Qiskit/qiskit-terra.git`
        - cd `qiskit-terra && git checkout origin/main`
        - `pip install .`
        - If while installing you receive an error that looks like
          ```
          ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qiskit 0.31.0 requires qiskit-terra==0.18.3, but you have qiskit-terra 0.19.0 which is incompatible.
          ```
          Please ignore this error, it is not actually an error but desired.
- Git clone or download the IEEE IBM Quantum tutorial repository. `git clone <insert>`. Make sure to download all necessary files.
    - Notebooks and extra files for this tutorial can be found [here](./). Navigate to this folder
- Launch Jupyter notebook `jupyter notebook` in [this](./) folder root.
- Open up this notebook "Hello OpenQASM3" and run to check your installation and run your first OpenQASM3 program on real hardware.

### Quantum Lab
*note*: As some features of this tutorial rely on code only available in Qiskit Terra's `main` branch not all portions of the demo will be supported in the Quantum lab. It is recommended that you follow the instructions for your local machine above.

Instructions:
- Open the [Quantum Lab](https://lab.quantum-computing.ibm.com/)
- In the sidebard upload the tutorial notebooks and all *.py files.

Please verify that your `qiskit-terra` version below is `0.19.0`.

In [2]:
import qiskit.tools.jupyter

%qiskit_version_table

/Users/thomas/opt/anaconda3/envs/openqasm3/lib/python3.9/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


- Launch this tutorial notebook from Jupyter in the tutorial root folder
- Great, you should be installed! Goto the "First Experiment" section

## Testing OpenQASM3 Support
Let's test out our installation and setup by verifying we have access to the code and hardware necessary.

Load our backend. We will be using `ibm_perth`, with a hub/group/project that has access to OpenQASM3 for today.

In [23]:
hub = 'ibm-q-community'
group = 'ieee-session'
project = 'event-2021'
backend_name = 'ibm_perth'

In [24]:
import qiskit
from qiskit import IBMQ
from qiskit.providers.ibmq.exceptions import IBMQAccountCredentialsNotFound

#IBMQ.save_account("<token>")
try:
    IBMQ.load_account()
except IBMQAccountCredentialsNotFound:
    print("Could not find your credentials"
          "Uncomment the lines above and insert your API token "
          "which can be found here https://quantum-computing.ibm.com/account."
          "After saving your credentials add back the comment."
         )
    raise

ibmqfactory.load_account:WARNING:2021-10-20 11:15:46,968: Credentials are already in use. The existing account in the session will be replaced.


In [25]:
from qiskit.providers.ibmq.exceptions import IBMQProviderError

# load provider
try:
    provider = IBMQ.get_provider(hub=hub, group=group, project=project)
except IBMQProviderError:
    print(
        "The provider was not found. "
        "This means your IBM Quantum Services account was not yet added. "
        "Contact one of the tutorial presenters to add you."
    )
    raise
    
# Get our backend
backend = provider.get_backend(backend_name)

Prepare our circuit:

In [26]:
qubit = 0

In [27]:
from qiskit import QuantumCircuit

h_qc = QuantumCircuit(1, 1, name="|+> Prep")
h_qc.h(0)
h_qc.measure(0, 0)
h_qc.draw(idle_wires=False)

┌───┐┌─┐
  q: ┤ H ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0

Transpile our quantum circuit for the hardware basis gates.

In [28]:
from qiskit import transpile

h_qc_transpiled = transpile(h_qc, backend, initial_layout=[1])
h_qc_transpiled.draw(idle_wires=False)

global phase: π/4
         ┌─────────┐┌────┐┌─────────┐┌─┐
q_0 -> 1 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤M├
         └─────────┘└────┘└─────────┘└╥┘
    c: 1/═════════════════════════════╩═
                                      0

Submit our circuit to the backend to verify its working.

In [29]:
job = backend.run(h_qc_transpiled)
print(f"Your job's id: {job.job_id()}")
result = job.result()
print(f"|+> preparation results: {result.get_counts(0)}")

Your job's id: 6170249e32bb439adaa548ed
|+> preparation results: {'0': 527, '1': 497}


In [30]:
job.error_message()

Run our circuit as a OpenQASM3 program

If you receive a permissions error or the job does not complete. Please post the error to one of the tutorial presenters for help debugging.

In [33]:
from run_openqasm3 import run_openqasm3
job = run_openqasm3(h_qc_transpiled, backend)

======|+> Prep======
=======QASM3======
OPENQASM 3;
bit[1] c;
rz(pi/2) $1;
sx $1;
rz(pi/2) $1;
c[0] = measure $1;

global phase: π/4
                                              
ancilla_0 -> 0 ───────────────────────────────
               ┌─────────┐┌────┐┌─────────┐┌─┐
      q_0 -> 1 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤M├
               └─────────┘└────┘└─────────┘└╥┘
ancilla_1 -> 2 ─────────────────────────────╫─
                                            ║ 
ancilla_2 -> 3 ─────────────────────────────╫─
                                            ║ 
ancilla_3 -> 4 ─────────────────────────────╫─
                                            ║ 
ancilla_4 -> 5 ─────────────────────────────╫─
                                            ║ 
ancilla_5 -> 6 ─────────────────────────────╫─
                                            ║ 
          c: 1/═════════════════════════════╩═
                                            0 
Running: c5o2aqlrd6ls96t2rhr0
=======|+> Prep=======
{'0': 520, '1':